In [ ]:
# enabling 3rd party widgets
# from google.colab import output
# output.enable_custom_widget_manager()
# output.disable_custom_widget_manager()

# interactive 3D plot
# !pip install ipympl
# %matplotlib widget

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

import time as time

import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import L2
import h5py

In [ ]:
colab_flag = False

FTYPE = np.float32
ITYPE = np.int32

array = np.array
float32 = np.float32
int32 = np.int32
float64 = np.float64
int64 = np.int64

In [ ]:
worker_id = 0

In [ ]:
if colab_flag == True:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Github/MLROM/KS/')

In [ ]:
print(os.getcwd())

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

if colab_flag == False:
    if gpus:
        gpu_to_use = 1
        tf.config.set_visible_devices(gpus[gpu_to_use], 'GPU')
        logical_devices = tf.config.list_logical_devices('GPU')
        print(logical_devices)

In [ ]:
from tools.misc_tools import create_data_for_RNN
from tools.ae_v5 import Autoencoder
from tools.ESN_v3 import ESN

In [ ]:
# print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())
print(tf.config.list_logical_devices())
print(tf.__version__)

# KS System

In [ ]:
gs_idx = '002'
dir_name_gs_all = os.getcwd() + '/grid_search/gridsearch_' + gs_idx

# worker_dir_list = os.listdir(dir_name_gs_all)
# worker_dir_list = [dir_name_gs_all+'/'+dname for dname in worker_dir_list if os.path.isdir(dir_name_gs_all+'/'+dname)]

worker_dir_list = [dir_name_gs_all+'/worker_{}'.format(worker_id)]

ESN_dir_list = []
for dname in worker_dir_list:
    temp = os.listdir(dname)
    temp = [dname+'/'+esndir for esndir in temp if os.path.isdir(dname+'/'+esndir)]
    ESN_dir_list.extend(temp)

analyse_only_if_not_already = False # This will check if the prediction horizons for the
                                    # for the chosen data have already been computed, and
                                    # and proceed only if they haven't. Setting it to `False`
                                    # will cause it to do the computations anyway and overwrite
                                    # any existing files.

In [ ]:
# setting up params (and saving, if applicable)

# RNN directory
dir_name_rnn = ESN_dir_list[0]

# reading AE directory
with open(dir_name_rnn + '/sim_data_AE_params.txt') as f:
    lines = f.readlines()

params_dict = eval(''.join(lines))

dir_name_ae = params_dict['dir_name_ae']
ae_idx = dir_name_ae[-3:]
dir_name_ae = os.getcwd()+'/saved_ae/ae_'+ae_idx
try:
    use_ae_data = params_dict['use_ae_data']
except:
    print("'use_ae_data' not present in sim_data_AE_params, set to True.")
    use_ae_data = True

# reading RNN paramaters
with open(dir_name_rnn + '/RNN_specific_data.txt') as f:
    lines = f.readlines()

params_rnn_dict = eval(''.join(lines))



dt_rnn = params_rnn_dict['dt_rnn']
T_sample_input = params_rnn_dict['T_sample_input']
T_sample_output = params_rnn_dict['T_sample_output']
T_offset = params_rnn_dict['T_offset']
return_params_arr = params_rnn_dict['return_params_arr']
params = params_rnn_dict['params']
delta_t = params_rnn_dict['delta_t']
normalize_dataset = params_rnn_dict['normalize_dataset']
num_input_tsteps = params_rnn_dict['num_input_tsteps']
skip_intermediate = params_rnn_dict['skip_intermediate']
normalization_type = params_rnn_dict['normalization_type']

normalization_arr_rnn = None
if normalize_dataset == True:
    with np.load(dir_name_rnn+'/normalization_data.npz', allow_pickle=True) as f:
        normalization_arr_rnn = f['normalization_arr'][0]
    
# reading simulation parameters
with open(dir_name_ae + '/ae_data.txt') as f:
    lines = f.readlines()
params_dict = eval(''.join(lines))
data_dir_idx = params_dict['data_dir_idx']
normalizeforae_flag = params_dict['normalizeforae_flag']

with np.load(dir_name_ae+'/normalization_data.npz', allow_pickle=True) as f:
    normalization_constant_arr_aedata = f['normalization_constant_arr_aedata'][0]

try:
    ae_data_with_params = params_dict['ae_data_with_params']
except:
    print("'ae_data_with_params' not present in ae_data, set to 'True'.")
    ae_data_with_params = True

print('dir_name_rnn:', dir_name_rnn)
print('dir_name_ae:', dir_name_ae)
print('data_dir_idx:', data_dir_idx)

# loading data
dir_name_data = os.getcwd() + '/saved_data/data_' + data_dir_idx
    
with open(dir_name_data + '/sim_data_params.txt') as f:
    lines = f.readlines()
params_dict = eval(''.join(lines))
params_mat = params_dict['params_mat']
# init_state_mat = params_dict['init_state_mat']
t0 = params_dict['t0']
T = params_dict['T']
delta_t = params_dict['delta_t']
numpoints_xgrid = params_dict['numpoints_xgrid']
length = params_dict['length']
return_params_arr = params_dict['return_params_arr']
normalize_flag_ogdata = params_dict['normalize_flag']
print('normalize_flag_ogdata:', normalize_flag_ogdata)
alldata_withparams_flag = params_dict['alldata_withparams_flag']

xgrid = length*np.linspace(0, 1, numpoints_xgrid)

with np.load(dir_name_data+'/data.npz', allow_pickle=True) as fl:
    all_data = fl['all_data']
    boundary_idx_arr = fl['boundary_idx_arr']
    normalization_constant_arr_ogdata = fl['normalization_constant_arr'][0]
    initial_t0 = fl['initial_t0']
    init_state_mat = fl['init_state_mat']
    lyapunov_spectrum_mat = fl['lyapunov_spectrum_mat']

# training params
with open(dir_name_rnn + '/training_specific_params.txt') as f:
    lines = f.readlines()

tparams_dict = eval(''.join(lines))

# learning_rate_dict = tparams_dict['learning_rate_dict']
epochs = tparams_dict['epochs']
patience = tparams_dict['patience']
min_delta = tparams_dict['min_delta']
prng_seed = tparams_dict['prng_seed']
train_split = tparams_dict['train_split']
val_split = tparams_dict['val_split']
batch_size = tparams_dict['batch_size']

test_split = 1 - train_split - val_split

# setting seed for PRNGs
np.random.seed(prng_seed)
tf.random.set_seed(prng_seed)

In [ ]:
lyapunov_time_arr = np.empty(shape=lyapunov_spectrum_mat.shape[0], dtype=FTYPE)
for i in range(lyapunov_spectrum_mat.shape[0]):
    lyapunov_time_arr[i] = 1/lyapunov_spectrum_mat[i, 0]
    print('Case : {}, lyapunov exponent : {}, lyapunov time : {}s'.format(i+1, lyapunov_spectrum_mat[i, 0], lyapunov_time_arr[i]))

In [ ]:
def plot(
        boundary_idx_arr,
        delta_t,
        all_data,
        xgrid,
        xticks_snapto=20,
        num_yticks=11,
    ):

    n = len(boundary_idx_arr)
    # '''
    num_cols = 1
    num_rows = 1
    factor = 1
    # fig = plt.figure(figsize=(7.5*num_cols, 7.5*num_rows))

    num_modes = xgrid.shape[0]

    prev_idx = 0
    for i in range(len(boundary_idx_arr)):
        next_idx = boundary_idx_arr[i]
        fig, ax = plt.subplots(figsize=(factor*7.5*num_cols, factor*5.0*num_rows))
        N = next_idx - prev_idx
        input_time = np.arange(0, N)*delta_t

        im = ax.imshow(all_data[prev_idx:next_idx, 0:num_modes].transpose(), aspect='auto', origin='lower')
        num_xticks = 1 + int((N*delta_t + 0.5*xticks_snapto) // xticks_snapto)
        # xticks = np.linspace(0, N, num_xticks, dtype=np.int32)
        xticks = np.arange(0, N, int((xticks_snapto+0.5*delta_t)//delta_t))
        ax.set_xticks(ticks=xticks)
        ax.set_xticklabels(np.round(xticks*delta_t, 1))
        ax.tick_params(axis='x', rotation=270+45)

        yticks = np.linspace(0, 1, num_yticks)*(len(xgrid)-1)
        yticklabels = np.round(np.linspace(0, 1, yticks.shape[0])*xgrid[-1], 2)
        ax.set_yticks(ticks=yticks)
        ax.set_yticklabels(yticklabels)

        ax.set_xlabel('Time')
        ax.set_ylabel(r'$x$')
        # ax.title.set_text(r'Latent States')

        plt.colorbar(im)
        plt.show()
        print('')

        prev_idx = next_idx

    # '''

In [ ]:
# dealing with normalizing the data before feeding into autoencoder
if normalizeforae_flag == True and use_ae_data == True:
    for i in range(numpoints_xgrid):
        all_data[:, i] -= normalization_constant_arr_aedata[0, i]
        all_data[:, i] /= normalization_constant_arr_aedata[1, i]
if ae_data_with_params == False:
    all_data = all_data[:, 0:numpoints_xgrid]

# Autoencoder

In [ ]:
# load_file = dir_name_ae+dir_sep+'new_class_data'+dir_sep+'ae_'+ae_idx+'_class_dict.txt'
# wt_file = dir_name_ae+dir_sep+'new_class_data'+dir_sep+'ae_'+ae_idx+'_ae_weights.h5'
if use_ae_data == True:
    load_file = dir_name_ae+'/final_net/final_net_class_dict.txt'
    wt_file = dir_name_ae+'/final_net/final_net_ae_weights.h5'

In [ ]:
if use_ae_data == True:
    ae_net = Autoencoder(all_data.shape[1], load_file=load_file)
    ae_net.load_weights_from_file(wt_file)

In [ ]:
# create data
if use_ae_data == True:
    latent_states_all = ae_net.encoder_net.predict(all_data)
    del(all_data)
else:
    latent_states_all = all_data[:, 0:xgrid.shape[0]]
num_latent_states = latent_states_all.shape[1]

time_stddev = np.std(latent_states_all, axis=0)

In [ ]:
# plot_latent_states_KS(
#     boundary_idx_arr,
#     latent_states_all,
#     delta_t,
#     dir_name_ae,
#     xticks_snapto=int(40*np.round((T//10)/40)),
#     num_yticks=11,
#     save_figs=False)

In [ ]:
# for i in range(ae_net.layers):
#     tf.keras.utils.plot_model(
#         ae_net.layers[i],
#         to_file=dir_name_ae+'/plots/netlayer_{}.png'.format(i),
#         show_shapes=True,
#         dpi=300
# )

# ESN

In [ ]:
T_sample_input = np.mean(lyapunov_time_arr)#50.1*dt_rnn
T_sample_output = 4*np.mean(lyapunov_time_arr)
T_offset = T_sample_input

In [ ]:
skip_intermediate = 1/2

In [ ]:
rnn_res_dict = create_data_for_RNN(
    latent_states_all,
    dt_rnn,
    T_sample_input,
    T_sample_output,
    T_offset,
    None,
    boundary_idx_arr,
    delta_t,
    params=params,
    return_numsamples=True,
    normalize_dataset=normalize_dataset,
    normalization_arr_external=normalization_arr_rnn,
    return_OrgDataIdxArr=False,
    normalization_type=normalization_type,
    skip_intermediate=skip_intermediate,
)
    
data_rnn_input = rnn_res_dict['data_rnn_input']
data_rnn_output = rnn_res_dict['data_rnn_output']
org_data_idx_arr_input = rnn_res_dict['org_data_idx_arr_input']
org_data_idx_arr_output = rnn_res_dict['org_data_idx_arr_output']
num_samples = rnn_res_dict['num_samples']
normalization_arr = rnn_res_dict['normalization_arr']
rnn_data_boundary_idx_arr = rnn_res_dict['rnn_data_boundary_idx_arr']

In [ ]:
del(org_data_idx_arr_input)
del(org_data_idx_arr_output)
del(latent_states_all)

In [ ]:
data_to_consider = 'testing' # could be 'all', 'training', 'testing', 'val'

In [ ]:
if data_to_consider != 'all':
    cum_samples = rnn_data_boundary_idx_arr[-1]
    num_train = 0
    num_val = 0
    begin_idx = 0
    for i in range(len(boundary_idx_arr)):
        num_samples = rnn_data_boundary_idx_arr[i] - begin_idx
        num_train += int( (1-test_split-val_split)*num_samples )
        num_val += int(val_split*num_samples)
        begin_idx = rnn_data_boundary_idx_arr[i]

    # defining shapes
    training_input_shape = [num_train]
    training_input_shape.extend(data_rnn_input.shape[1:])

    training_output_shape = [num_train]
    training_output_shape.extend(data_rnn_output.shape[1:])

    val_input_shape = [num_val]
    val_input_shape.extend(data_rnn_input.shape[1:])

    val_output_shape = [num_train]
    val_output_shape.extend(data_rnn_output.shape[1:])

    testing_input_shape = [cum_samples-num_train-num_val]
    testing_input_shape.extend(data_rnn_input.shape[1:])

    testing_output_shape = [cum_samples-num_train-num_val]
    testing_output_shape.extend(data_rnn_output.shape[1:])

    shape_to_use = eval(data_to_consider+'_input_shape')
    rnn_data_idx = np.empty(shape=shape_to_use[0], dtype=np.int32)
    
    begin_idx = 0
    training_data_rolling_count = 0
    val_data_rolling_count = 0
    testing_data_rolling_count = 0
    for i in range(len(boundary_idx_arr)):
        num_samples = rnn_data_boundary_idx_arr[i] - begin_idx
        num_train = int( (1-test_split-val_split)*num_samples )
        num_val = int(val_split*num_samples)
        num_test = num_samples-num_train-num_val+1

        if data_to_consider == 'training':
            rnn_data_idx[training_data_rolling_count:training_data_rolling_count+num_train] = np.arange(begin_idx, begin_idx+num_train)
        elif data_to_consider == 'val':
            rnn_data_idx[val_data_rolling_count:val_data_rolling_count+num_val] = np.arange(begin_idx+num_train, begin_idx+num_train+num_val)
        elif data_to_consider == 'testing':
            rnn_data_idx[testing_data_rolling_count:testing_data_rolling_count+num_test] = np.arange(begin_idx+num_train+num_val, rnn_data_boundary_idx_arr[i])

        training_data_rolling_count += num_train
        val_data_rolling_count += num_val
        testing_data_rolling_count += num_test

        begin_idx = rnn_data_boundary_idx_arr[i]

    # shuffling
    np.random.shuffle(rnn_data_idx)
    data_rnn_input = data_rnn_input[rnn_data_idx]
    data_rnn_output = data_rnn_output[rnn_data_idx]
    del(rnn_data_idx)

In [ ]:
# del(data_rnn_input)
# del(data_rnn_output)

In [ ]:
print(' data_rnn_input.shape :', data_rnn_input.shape)
print('data_rnn_output.shape :', data_rnn_output.shape)

In [ ]:
# idx = np.arange(data_rnn_input.shape[0])
# np.random.shuffle(idx)
# boundary = int(np.round(train_split*data_rnn_input.shape[0]))

# training_data_rnn_input = data_rnn_input[idx[0:boundary]]
# training_data_rnn_output = data_rnn_output[idx[0:boundary]]

# testing_data_rnn_input = data_rnn_input[idx[boundary:]]
# testing_data_rnn_output = data_rnn_output[idx[boundary:]]

In [ ]:
def rescale_data(data, normalization_arr):
    '''
    data - [num_batches x num_timesteps x num_states]
    normalization_arr = [2 x num_states]
    '''
    new_data = data.copy()
    shape = new_data.shape
    for i in range(data.shape[-1]):
        new_data[:, i] -= normalization_arr[0, i]
        new_data[:, i] /= normalization_arr[1, i]

    return new_data

def norm_sq_time_average(data):
    data_norm_sq = np.zeros(shape=data.shape[0])
    for i in range(data.shape[1]):
        data_norm_sq[:] += data[:, i]**2
    # integrating using the trapezoidal rule
    norm_sq_time_avg = np.sum(data_norm_sq) - 0.5*(data_norm_sq[0]+data_norm_sq[-1])
    norm_sq_time_avg /= data_norm_sq.shape[0]
    return norm_sq_time_avg

def invert_normalization(data, normalization_arr):
    new_data = data.copy()
    shape = new_data.shape
    for i in range(shape[-1]):
        if len(shape) == 2:
            new_data[:, i] *= normalization_arr[1, i]
            new_data[:, i] += normalization_arr[0, i]
        elif len(shape) == 3:
            new_data[:, :, i] *= normalization_arr[1, i]
            new_data[:, :, i] += normalization_arr[0, i]
    return new_data

In [ ]:
def plot_histogram_and_save(
    prediction_horizon_arr,
    save_dir,
    savefig_fname='pre_ARtraining',
    bin_width=0.05,
    bin_begin=0.0):
    
    fig, ax = plt.subplots()

    ph_mean = np.mean(prediction_horizon_arr)
    ph_stddev = np.std(prediction_horizon_arr)
    ph_max = np.max(prediction_horizon_arr)
    ph_min = np.min(prediction_horizon_arr)
    
    bin_end = bin_width*np.round((np.max(prediction_horizon_arr)+0.5*bin_width)//bin_width)
    nbins = int(np.round(bin_end/bin_width))

    ax.hist(prediction_horizon_arr, bins=nbins, range = [bin_begin, bin_end], density=True)
    ax.axvline(ph_mean, linewidth=0.9, linestyle='--', color='k')

    ax.set_xlabel('Prediction Horizon (Lyapunov times)')
    ax.set_ylabel('PDF')

    ax.grid(True)
    # ax.set_axisbelow(True)

    ax.text(
        0.01 + ax.transAxes.inverted().transform(ax.transData.transform([prediction_horizon, 0]))[0],
        0.8,
        'mean',
        rotation=90,
        verticalalignment='bottom',
        horizontalalignment='left',
        bbox=dict(facecolor=np.array([255,255,153])/255, alpha=1, boxstyle='square,pad=0.2'),
        transform=ax.transAxes
    )

    text_xy = [0.95, 0.95]
    ax.text(
        text_xy[0],
        text_xy[1],
        'mean : {:.4f}\nmax : {:.4f}\nmin : {:.4f}\nstddev : {:.4f}'.format(
            ph_mean,
            ph_max,
            ph_min,
            ph_stddev,
        ),
        transform=ax.transAxes,
        bbox=dict(
            boxstyle="round",
            ec=(0.6, 0.6, 1),
            fc=(0.9, 0.9, 1),
        ),
        # bbox=dict(facecolor='C0', alpha=0.5, boxstyle='round,pad=0.2'),
        horizontalalignment='right',
        verticalalignment='top'
    )

    ax.set_title('nbins = {}'.format(nbins))
    
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)

    fig.savefig(save_dir+'/'+savefig_fname+'.png', dpi=300, bbox_inches='tight')
    fig.clear()
    plt.close()

In [ ]:
data_in_og = data_rnn_input
data_out_og = data_rnn_output

num_runs = 100
num_runs = np.min([num_runs, data_in_og.shape[0]])
print('num_runs : {}\n'.format(num_runs))
error_threshold = 0.5

# data_idx_arr = np.arange(data_in_og.shape[0])
# np.random.shuffle(data_idx_arr)
data_idx_arr = np.linspace(0, data_in_og.shape[0]-1, num_runs, dtype=np.int32)

ph_mat = np.empty(shape=(len(ESN_dir_list), num_runs))

savefig_fname = 'pre_ARtraining-'+data_to_consider+'data'
npsavedata_fname = '/prediction_horizons-'+data_to_consider+'data'
plot_dir = '/plots'

for dname_idx in range(len(ESN_dir_list)):
    analysis_time = time.time()
    dir_name_rnn = ESN_dir_list[dname_idx]
    
    sidx1 = dir_name_rnn[::-1].index('/')
    sidx2 = dir_name_rnn[-sidx1-2::-1].index('/')
    print(dir_name_rnn[-(sidx1+sidx2+1):])
    print('{} / {}'.format(dname_idx+1, len(ESN_dir_list)))
    
    if analyse_only_if_not_already == True:
        # checking if prediction horizons have already been computed
        check1 = os.path.exists(dir_name_rnn+plot_dir)
        check2 = os.path.exists(dir_name_rnn+plot_dir+'/'+savefig_fname+'.png')
        check3 = os.path.exists(dir_name_rnn+npsavedata_fname+'.npz')
        if check1 and check2 and check3:
            # skip this directory
            print('prediction horizons have been computed, skipping this dir\n')
            with np.load(dir_name_rnn+npsavedata_fname+'.npz') as fl:
                prediction_horizon_arr = fl['prediction_horizon_arr']
            ph_mat[dname_idx, :] = prediction_horizon_arr
            continue

    # Initialize network
    load_file = dir_name_rnn + '/final_net/final_net_class_dict.txt'
    rnn_net = ESN(
        load_file=load_file,
        stddev=0.0,
    )
    rnn_net.build(input_shape=(1, 1, data_rnn_input.shape[-1]))

    wt_file = dir_name_rnn+'/final_net/final_net_ESN_weights.hdf5'
    rnn_net.load_weights_from_file(wt_file)

    prediction_horizon_arr = np.empty(shape=num_runs)

    for i in range(num_runs):
        data_idx = data_idx_arr[i]

        for j in range(len(rnn_data_boundary_idx_arr)):
            if data_idx < rnn_data_boundary_idx_arr[j]:
                case_idx = j
                break
        lyap_time = lyapunov_time_arr[j]

        data_out = data_out_og[data_idx]
        # data_out = rescale_data(data_out, normalization_arr)
        data_out = invert_normalization(data_out, normalization_arr)

        prediction_lst = []

        for layer in rnn_net.ESN_layers:
            layer.reset_states()

        input_preds = np.array(rnn_net(data_in_og[data_idx:data_idx+1, :, :], training=False))[0]

        prediction_lst.append(input_preds[-1])

        for k in range(1, data_out.shape[-2]):
            data_in = np.array([[prediction_lst[-1]]])
            output = np.array(rnn_net(data_in, training=False))[0, 0]
            prediction_lst.append(output)
        prediction_lst = np.stack(prediction_lst)
        prediction_lst = invert_normalization(prediction_lst, normalization_arr)
            
        ### Error and prediction horizon
        # error = np.linalg.norm(data_out[:, :] - prediction[i, :, :], axis=1)
        error = (data_out[:, :] - prediction_lst[:, :])**2
        # error /= norm_sq_time_average(data_out)**0.5
        error = np.mean(np.divide(error, time_stddev**2), axis=1)**0.5

        predhor_idx = np.where(error >= error_threshold)[0]
        if predhor_idx.shape[0] == 0:
            predhor_idx = error.shape[0]
        else:
            predhor_idx = predhor_idx[0]

        prediction_horizon_arr[i] = predhor_idx*dt_rnn/lyap_time
    
    ph_mat[dname_idx, :] = prediction_horizon_arr

    prediction_horizon = np.mean(prediction_horizon_arr)
    stddev_ph = np.std(prediction_horizon_arr)
    
    plot_histogram_and_save(
        prediction_horizon_arr,
        save_dir=dir_name_rnn+plot_dir,
        savefig_fname=savefig_fname
    )
    
    np.savez(
        dir_name_rnn+npsavedata_fname,
        prediction_horizon_arr=prediction_horizon_arr
    )
    
    print('prediction_horizon : {}, stddev_ph : {}'.format(prediction_horizon, stddev_ph))
    print('analysis time : {} s\n'.format(time.time() - analysis_time))

In [ ]:
np.savez(
    worker_dir_list[0]+'/prediction_horizon_mat',
    dir_list=[dname[-sidx1:] for dname in ESN_dir_list],
    ph_mat=ph_mat
)

In [ ]:
ph_mean_arr = np.mean(ph_mat, axis=1)
idx = np.where(np.max(ph_mean_arr) == ph_mean_arr)[0][0]
ESN_dir = ESN_dir_list[idx]

load_file = ESN_dir + '/final_net/final_net_class_dict.txt'
with open(load_file, 'r') as f:
    lines = f.readlines()
load_dict = eval(lines[0])

ESN_layers_units = load_dict['ESN_layers_units']
omega_in = load_dict['omega_in']
sparsity = load_dict['sparsity']
rho_res = load_dict['rho_res']
usebias_Win = load_dict['usebias_Win']
alpha = load_dict['alpha']
usebias_Wout = load_dict['usebias_Wout']
degree_of_connectivity = [int((ESN_layers_units[i]-1)*(1-sparsity[i])) for i in range(len(sparsity))]

print('max ph_mean : {}'.format(ph_mean_arr[idx]))
print(ESN_dir)

print('ESN_layers_units : {}'.format(ESN_layers_units))
print('omega_in : {}'.format(omega_in))
print('sparsity : {}'.format(sparsity))
print('degree_of_connectivity : {}'.format(degree_of_connectivity))
print('rho_res : {}'.format(rho_res))
print('usebias_Win : {}'.format(usebias_Win))
print('alpha : {}'.format(alpha))
print('usebias_Wout : {}'.format(usebias_Wout))